In [21]:
!pip uninstall -y datasets
!pip install datasets==2.18.0  # Stable older version known to work in Colab


Found existing installation: datasets 2.14.4
Uninstalling datasets-2.14.4:
  Successfully uninstalled datasets-2.14.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.9/170.9 kB 20.0 MB/s eta 0:00:00
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 447, in run
^C


In [ ]:
from datasets import load_dataset

dataset = load_dataset("scientific_papers", "pubmed")

# Save explicitly to a local folder (in Colab's memory)
dataset.save_to_disk("/content/pubmed_dataset")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/datasets/load.py:1461: FutureWarning: The repository for scientific_papers contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/scientific_papers
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warni

Generating train split:   0%|          | 0/119924 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6633 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6658 [00:00<?, ? examples/s]

Saving the dataset (0/5 shards):   0%|          | 0/119924 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/6633 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/6658 [00:00<?, ? examples/s]

In [1]:
from datasets import load_from_disk

dataset = load_from_disk("/content/pubmed_dataset")
print(dataset)
print(dataset["train"][0])


ModuleNotFoundError: No module named 'datasets'

In [ ]:
from datasets import load_from_disk
from transformers import AutoTokenizer

dataset = load_from_disk("/content/pubmed_dataset")
tokenizer = AutoTokenizer.from_pretrained("google/bigbird-pegasus-large-pubmed")

# Tokenization function
def preprocess(example):
    model_inputs = tokenizer(
        example["article"], max_length=4096, truncation=True
    )
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            example["abstract"], max_length=256, truncation=True
        )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply tokenization (this might take time, consider using `batched=True`)
tokenized_dataset = dataset.map(preprocess, batched=True)

# Save tokenized dataset
tokenized_dataset.save_to_disk("/content/tokenized_pubmed")


Map:   0%|          | 0/119924 [00:00<?, ? examples/s]

Map:   0%|          | 0/6633 [00:00<?, ? examples/s]

Map:   0%|          | 0/6658 [00:00<?, ? examples/s]

Saving the dataset (0/9 shards):   0%|          | 0/119924 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/6633 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/6658 [00:00<?, ? examples/s]

In [20]:
!pip install transformers peft datasets accelerate


In [ ]:
from transformers import AutoModelForSeq2SeqLM
from peft import LoraConfig, get_peft_model
import torch

# Load base model
model = AutoModelForSeq2SeqLM.from_pretrained("google/bigbird-pegasus-large-pubmed")

# Define LoRA configuration
peft_config = LoraConfig(
    task_type="SEQ_2_SEQ_LM",
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["q_proj", "v_proj", "k_proj", "out_proj"],
    bias="none"
)

# Apply LoRA
model = get_peft_model(model, peft_config)

# Print number of trainable parameters
model.print_trainable_parameters()


trainable params: 2,097,152 || all params: 578,989,056 || trainable%: 0.3622


In [16]:
!pip uninstall -y transformers
!pip install transformers==4.36.2 peft datasets accelerate


Found existing installation: transformers 4.51.3
Uninstalling transformers-4.51.3:
  Successfully uninstalled transformers-4.51.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 129.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 109.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 107.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 68.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.1 MB/s eta 

In [2]:
!pip uninstall -y peft
!rm -rf /usr/local/lib/python3.11/dist-packages/peft



Found existing installation: peft 0.7.1
Uninstalling peft-0.7.1:
  Successfully uninstalled peft-0.7.1


In [3]:
!pip install peft==0.7.1



  Using cached peft-0.7.1-py3-none-any.whl.metadata (25 kB)
Using cached peft-0.7.1-py3-none-any.whl (168 kB)


In [19]:
!pip uninstall -y accelerate
!pip install accelerate==0.24.1


Found existing installation: accelerate 1.6.0
Uninstalling accelerate-1.6.0:
  Successfully uninstalled accelerate-1.6.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 19.2 MB/s eta 0:00:00


In [ ]:
import os
os._exit(0)


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from peft import LoraConfig, get_peft_model

tokenizer = AutoTokenizer.from_pretrained("google/bigbird-pegasus-large-pubmed")
model = AutoModelForSeq2SeqLM.from_pretrained("google/bigbird-pegasus-large-pubmed")

peft_config = LoraConfig(
    task_type="SEQ_2_SEQ_LM",
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["q_proj", "v_proj", "k_proj", "out_proj"],
    bias="none"
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()


/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your 

trainable params: 2,097,152 || all params: 578,989,056 || trainable%: 0.3622092642801179


In [ ]:
from datasets import load_from_disk
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments

# Load the tokenized dataset
dataset = load_from_disk("/content/tokenized_pubmed")

# Use a subset for training (faster)
train_dataset = dataset["train"].select(range(20000))
val_dataset = dataset["validation"]

# Data collator for seq2seq
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, padding="longest", return_tensors="pt")

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./lora_bigbird_pubmed",
    evaluation_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    logging_steps=100,
    save_steps=1000,
    save_total_limit=2,
    bf16=True,
    report_to="none"
)

# Initialize trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator
)

# 🔥 Start training
trainer.train()

# 💾 Save the LoRA adapter
trainer.model.save_pretrained("lora_bigbird_pubmed_adapter")


You're using a PegasusTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,1.077500,2.092253


Attention type 'block_sparse' is not possible if sequence_length: 181 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...


In [2]:
from peft import PeftConfig

adapter_path = "/content/drive/MyDrive/fine_tuned_biogpt_lora"

# ✅ This is the correct way to load configs with new fields like 'alpha_pattern'
peft_config = PeftConfig.from_pretrained(adapter_path)


TypeError: LoraConfig.__init__() got an unexpected keyword argument 'corda_config'

In [1]:
import json
from transformers import AutoModelForSeq2SeqLM
from peft import PeftModel, PeftConfig

# Define path to your adapter directory
adapter_path = "/content/drive/MyDrive/fine_tuned_biogpt_lora"

# Step 1: Manually load adapter config
with open(f"{adapter_path}/adapter_config.json") as f:
    config_dict = json.load(f)

# Step 2: Create PEFT config object
peft_config = PeftConfig(**config_dict)

# Step 3: Load the base BigBird-Pegasus model
base_model = AutoModelForSeq2SeqLM.from_pretrained(peft_config.base_model_name_or_path)

# Step 4: Attach the LoRA adapter
model = PeftModel(base_model, peft_config)
model.load_adapter(adapter_path)
model = model.eval()


/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


TypeError: PeftConfig.__init__() got an unexpected keyword argument 'alpha_pattern'

In [5]:
def summarize_article(text, max_summary_length=256):
    inputs = tokenizer(text, return_tensors="pt", truncation=True)

    if model.device.type == "cuda" or hasattr(model, "cuda"):
        inputs = {k: v.to(model.device) for k, v in inputs.items()}

    summary_ids = model.generate(
        **inputs,
        max_length=max_summary_length,
        num_beams=4,
        early_stopping=True,
        no_repeat_ngram_size=3,
        length_penalty=1.0,
        repetition_penalty=1.2
    )

    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)


In [ ]:
from datasets import load_from_disk

dataset = load_from_disk("/content/tokenized_pubmed")
original_dataset = load_from_disk("/content/pubmed_dataset")  # raw texts

sample_article = original_dataset["validation"][0]["article"]
generated_summary = summarize_article(sample_article)

print("📄 Article:\n", sample_article[:1000], "...\n")
print("🧠 Generated Summary:\n", generated_summary)


📄 Article:
 approximately , one - third of patients with symptomatic vte manifests pe , whereas two - thirds manifest dvt alone .
both dvt and pe can be clinically silent ( asymptomatic ) and hence not suspected .
if undiagnosed , asymptomatic vte can lead to chronic venous disease or recurrent vte and long - term debilitating sequelae such as postthrombotic syndrome and chronic thromboembolic pulmonary hypertension .
vte is not only disabling but also prolongs hospital stay and increases the cost of treatment .
along with myocardial infarction and arrhythmia ( due to electrolyte imbalance ) , pe is one of the commonest causes of sudden unexplained deaths in hospitalized patients .
it is estimated that 20 million cases of lower extremity dvt occur in the usa alone .
the prevailing notion that the incidence of vte in asians is less than that in the western population has been disproved by recent studies .
the incidence of postoperative dvt in indian patients undergoing major lower limb 

In [ ]:
!pip install sentence-transformers faiss-cpu


In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from peft import LoraConfig, get_peft_model, PeftModel

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("google/bigbird-pegasus-large-pubmed")

# Load base model
base_model = AutoModelForSeq2SeqLM.from_pretrained("google/bigbird-pegasus-large-pubmed")

# Load LoRA adapter
model = PeftModel.from_pretrained(base_model, "lora_bigbird_pubmed_adapter")
model = model.eval()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.92M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.51M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/775 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.31G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.31G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/232 [00:00<?, ?B/s]

ValueError: Can't find 'adapter_config.json' at 'lora_bigbird_pubmed_adapter'

In [ ]:
from datasets import load_from_disk
from tqdm import tqdm

raw_dataset = load_from_disk("/content/pubmed_dataset")
train_articles = raw_dataset["train"].select(range(100))  # You can increase this number

all_summaries = []

for item in tqdm(train_articles, desc="Generating summaries"):
    article_text = item["article"]
    summary_text = summarize_article(article_text)
    all_summaries.append(summary_text)

# Save summaries to disk
with open("generated_summaries.txt", "w", encoding="utf-8") as f:
    for summary in all_summaries:
        f.write(summary.strip() + "\n")


Generating summaries: 100%|██████████| 100/100 [1:07:56<00:00, 40.76s/it]


In [1]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

# Load SPECTER model
embedder = SentenceTransformer("sentence-transformers/allenai-specter")

# Load summaries from file
with open("generated_summaries.txt", "r", encoding="utf-8") as f:
    summaries = [line.strip() for line in f.readlines()]

# Encode summaries
summary_embeddings = embedder.encode(summaries, batch_size=16, show_progress_bar=True)
embeddings_array = np.array(summary_embeddings, dtype="float32")

# Normalize for cosine similarity
faiss.normalize_L2(embeddings_array)


ModuleNotFoundError: No module named 'faiss'

In [8]:
dim = embeddings_array.shape[1]
index = faiss.IndexFlatIP(dim)  # IP + normalized vectors = cosine sim

index.add(embeddings_array)

# Save index and summaries
faiss.write_index(index, "summary_index.faiss")

import pickle
with open("summaries_list.pkl", "wb") as f:
    pickle.dump(summaries, f)


NameError: name 'embeddings_array' is not defined

In [ ]:
query = "What are the causes of pulmonary embolism?"
query_vec = embedder.encode([query], normalize_embeddings=True)
query_vec = np.array(query_vec, dtype='float32')

D, I = index.search(query_vec, k=5)

print("🔍 Top Retrieved Summaries:\n")
for idx in I[0]:
    print(f"— {summaries[idx][:300]}...\n")


🔍 Top Retrieved Summaries:

— we used results from a national tuberculosis prevalence survey in eritrea to calculate case detection rate ( cdr ) and compared it with the cdr provided by the world health organization . although the incidence of smear - positive tuberculosis ( tb ) was underreported , the cdrs obtained may be an u...

— syncope is caused by transient diffuse cerebral hypoperfusion and is characterized by transient loss of consciousness with a rapid onset followed by spontaneous and complete recovery . significant fluctuations in cerebral perfusion pressure are prevented by autoregulation of cerebral circulation , b...

— the revised international classification of functioning and disability ( icf ) provides a framework for describing the complete range of functional states that capture the complete experience of health for all people , including persons with functional problems . icf provides the basis for the metho...

— t cells tell macrophages when to start making the

In [ ]:
!pip install transformers accelerate


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load BioGPT
qa_tokenizer = AutoTokenizer.from_pretrained("microsoft/biogpt")
qa_model = AutoModelForCausalLM.from_pretrained("microsoft/biogpt").eval()

def answer_question(question, context):
    prompt = f"Question: {question}\nContext: {context}\nAnswer:"
    inputs = qa_tokenizer(prompt, return_tensors="pt")

    with torch.no_grad():
        output = qa_model.generate(
            **inputs,
            max_length=256,
            num_beams=5,
            early_stopping=True
        )

    return qa_tokenizer.decode(output[0], skip_special_tokens=True)


pytorch_model.bin:   0%|          | 0.00/1.56G [00:00<?, ?B/s]

In [ ]:
def answer_question(question, context):
    # Manually truncate long context before tokenizing
    max_prompt_length = 3000  # chars, not tokens — adjust as needed
    prompt = f"Question: {question}\nContext: {context[:max_prompt_length]}\nAnswer:"

    # Now safely tokenize
    inputs = qa_tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=1024)

    try:
        with torch.no_grad():
            output = qa_model.generate(
                **inputs,
                max_new_tokens=128,
                num_beams=5,
                early_stopping=True,
                no_repeat_ngram_size=3
            )
        return qa_tokenizer.decode(output[0], skip_special_tokens=True)
    except IndexError:
        return "❌ Error: Input too long for BioGPT."


In [ ]:
# Combine top-5 summaries
top_k = 5
combined_context = " ".join([summaries[i] for i in I[0][:top_k]])

# Run QA on the combined context
question = "What are the causes of pulmonary embolism?"
answer = answer_question(question, combined_context)

print("🧠 Question:", question)
print("📄 Combined Context Sample:\n", combined_context[:500], "...")
print("✅ Answer:", answer)


🧠 Question: What are the causes of pulmonary embolism?
📄 Combined Context Sample:
 we used results from a national tuberculosis prevalence survey in eritrea to calculate case detection rate ( cdr ) and compared it with the cdr provided by the world health organization . although the incidence of smear - positive tuberculosis ( tb ) was underreported , the cdrs obtained may be an underestimation . syncope is caused by transient diffuse cerebral hypoperfusion and is characterized by transient loss of consciousness with a rapid onset followed by spontaneous and complete recovery  ...
✅ Answer: Question: What are the causes of pulmonary embolism? Context: we used results from a national tuberculosis prevalence survey in eritrea to calculate case detection rate (cdr) and compared it with the cdr provided by the world health organization. although the incidence of smear - positive tuberculosis (tb) was underreported, the cdrs obtained may be an underestimation. syncope is caused by transient

In [ ]:
!pip install evaluate

import evaluate

# Load EM and F1 metrics
exact_match = evaluate.load("exact_match")
f1_metric = evaluate.load("f1")


In [ ]:
predicted_answers = [
    "Pulmonary embolism is caused by blood clots that travel to the lungs.",
    "Pulmonary embolism is more common in stroke patients."
]

ground_truth_answers = [
    "Blood clots that block pulmonary arteries.",
    "Patients with stroke have a higher risk of pulmonary embolism."
]


In [ ]:
squad_metric = evaluate.load("squad")
results = squad_metric.compute(
    predictions=[{"id": str(i), "prediction_text": p} for i, p in enumerate(predicted_answers)],
    references=[{"id": str(i), "answers": {"text": [a], "answer_start": [0]}} for i, a in enumerate(ground_truth_answers)]
)

print("📊 QA Evaluation Metrics:")
print("✅ F1:", results["f1"])
print("✅ Exact Match:", results["exact_match"])



📊 QA Evaluation Metrics:
✅ F1: 47.05882352941176
✅ Exact Match: 0.0


In [3]:
!pip install streamlit
!pip install streamlit pyngrok


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 74.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 144.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.4 MB/s eta 0:00:00


In [5]:
%%writefile biomedical_qa_app.py
# (Paste the full Streamlit code block here)
import streamlit as st
import pickle
import faiss
import numpy as np
import torch
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# Load models
embedder = SentenceTransformer("sentence-transformers/allenai-specter")
qa_tokenizer = AutoTokenizer.from_pretrained("microsoft/biogpt")
qa_model = AutoModelForCausalLM.from_pretrained("microsoft/biogpt").eval()

# Load FAISS index and summaries
index = faiss.read_index("summary_index.faiss")
with open("summaries_list.pkl", "rb") as f:
    summaries = pickle.load(f)

# Define QA function
def answer_question(question, context):
    prompt = f"Question: {question}\nContext: {context}\nAnswer:"
    inputs = qa_tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024)
    with torch.no_grad():
        output = qa_model.generate(
            **inputs,
            max_new_tokens=128,
            num_beams=5,
            early_stopping=True,
            no_repeat_ngram_size=3
        )
    return qa_tokenizer.decode(output[0], skip_special_tokens=True)

# UI
st.title("🧠 Biomedical QA Assistant")
question = st.text_input("Ask a biomedical question")


if question:
    # Step 1: FAISS search
    query_vec = embedder.encode([question], normalize_embeddings=True)
    query_vec = np.array(query_vec, dtype='float32')
    D, I = index.search(query_vec, k=5)

    # Step 2: Filter by similarity threshold
    threshold = 0.4
    filtered_summaries = [
        summaries[i] for score, i in zip(D[0], I[0]) if score >= threshold
    ]

    if not filtered_summaries:
        answer = "No relevant context found to answer this question."
    else:
        combined_context = " ".join(filtered_summaries)
        answer = answer_question(question, combined_context)

    # Step 3: Display result
    st.subheader("✅ Answer")
    st.write(answer)

    st.subheader("📄 Top Retrieved Summaries")
    for i, summary in enumerate(filtered_summaries):
        st.markdown(f"**Summary {i+1}:** {summary[:500]}...")


Writing biomedical_qa_app.py


In [6]:
from pyngrok import ngrok
ngrok.set_auth_token("2x1Cl8bh0Jppk9gpeXhkAZPyl25_5CtV7c5Zvau9A1YVjzNVW")



In [7]:
from pyngrok import ngrok

# Kill any existing streamlit processes
!pkill streamlit
!pkill ngrok


# Run streamlit app
!streamlit run biomedical_qa_app.py &>/content/log.txt &

# Expose the app via ngrok
public_url = ngrok.connect(8501, "http")
print(f"🌐 App URL: {public_url}")


🌐 App URL: NgrokTunnel: "https://bb17-104-199-148-220.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
!pip install datasets

from datasets import load_dataset
pubmedqa = load_dataset("pubmed_qa", "pqa_labeled")


Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
example = pubmedqa["train"][0]
print("❓ Question:", example["question"])
print("📄 Context:", example["context"])
print("🧠 Answer:", example["long_answer"])


❓ Question: Do mitochondria play a role in remodelling lace plant leaves during programmed cell death?
📄 Context: {'contexts': ['Programmed cell death (PCD) is the regulated death of cells within an organism. The lace plant (Aponogeton madagascariensis) produces perforations in its leaves through PCD. The leaves of the plant consist of a latticework of longitudinal and transverse veins enclosing areoles. PCD occurs in the cells at the center of these areoles and progresses outwards, stopping approximately five cells from the vasculature. The role of mitochondria during PCD has been recognized in animals; however, it has been less studied during PCD in plants.', 'The following paper elucidates the role of mitochondrial dynamics during developmentally regulated PCD in vivo in A. madagascariensis. A single areole within a window stage leaf (PCD is occurring) was divided into three areas based on the progression of PCD; cells that will not undergo PCD (NPCD), cells in early stages of PCD (

In [ ]:
import evaluate

squad_metric = evaluate.load("squad")

predicted = []
references = []

for i in range(100):  # First 100 PubMedQA pairs
    q = pubmedqa["train"][i]["question"]
    ref = pubmedqa["train"][i]["long_answer"]

    # FAISS retrieval
    query_vec = embedder.encode([q], normalize_embeddings=True)
    query_vec = np.array(query_vec, dtype='float32')
    D, I = index.search(query_vec, k=5)

    # Filter top summaries by score
    threshold = 0.4
    filtered = [
        summaries[j] for score, j in zip(D[0], I[0]) if score >= threshold
    ]
    if not filtered:
        pred = "No relevant context found."
    else:
        context = " ".join(filtered)
        pred = answer_question(q, context)

    predicted.append({"id": str(i), "prediction_text": pred})
    references.append({"id": str(i), "answers": {"text": [ref], "answer_start": [0]}})


In [ ]:
results = squad_metric.compute(predictions=predicted, references=references)
print("📊 PubMedQA Evaluation on 100 Examples:")
print("✅ F1:", results["f1"])
print("✅ Exact Match:", results["exact_match"])


📊 PubMedQA Evaluation on 100 Examples:
✅ F1: 7.518624017007808
✅ Exact Match: 0.0
